In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [59]:
df = pd.read_csv('../data/eda.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,...,red_246,red_247,red_248,red_249,red_250,red_251,red_252,red_253,red_254,red_255
0,0,0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,...,1.0,3.0,1.0,2.0,2.0,4.0,2.0,0.0,2.0,31.0
1,1,1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,...,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,5.0


In [57]:
numbers = df.loc[:,'0':'255'].columns
rename_1 = dict(zip(df.loc[:,'256':'511'].columns, numbers))
rename_2 = dict(zip(df.loc[:,'512':'767'].columns, numbers))

blue = df.loc[:,'0':'255'].add_prefix('blue_')

df.loc[:,'256':'511'].rename(rename_1, axis=1, inplace=True)
green = df.loc[:,'0':'255'].add_prefix('green_')

df.loc[:,'512':'767'].rename(rename_2, axis=1, inplace=True)
red = df.loc[:,'0':'255'].add_prefix('red_')

df = pd.concat([df.iloc[:,0:9],blue,green,red], axis=1)
df.head()

,Unnamed: 0,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,blue_0,...,red_246,red_247,red_248,red_249,red_250,red_251,red_252,red_253,red_254,red_255
0,0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,0.0,...,1.0,3.0,1.0,2.0,2.0,4.0,2.0,0.0,2.0,31.0
1,1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,0.0,...,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,5.0


In [58]:
df.to_csv('../data/eda.csv')

In [50]:
df.loc[:,'0':'255'].add_prefix('blue_')

TypeError: add_prefix() got an unexpected keyword argument 'inplace'